In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma


In [2]:
%run /Users/enzo-macmini/auction_alg_ITU/parallel_auction_ITU.ipynb

In [3]:
def solve_1to1(Φ_i_j):
    n, m = np.shape(Φ_i_j)
    M_z_a = spr.bmat([[spr.kron(spr.eye(n), np.ones((1, m)))],
                      [spr.kron(np.ones((1, n)), spr.eye(m))]])
  
    q = np.concatenate((np.ones(n), np.ones(m)))

    model = grb.Model()
    mu_a = model.addMVar(n * m, vtype=grb.GRB.INTEGER)
    model.setObjective(Φ_i_j.flatten() @ mu_a, GRB.MAXIMIZE)
    model.addConstr(M_z_a @ mu_a <= q)
    
    model.Params.OutputFlag = 0
    model.optimize()

    return np.array(model.x, dtype= bool).reshape([n,m])*1

In [1343]:
solve_1to1(np.array([[ True , True],
 [ True , True],
 [ True , True]]))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


array([[1, 0],
       [0, 1],
       [0, 0]])

In [19]:
def iteration(self,i_ent, mu_, V_j_, perm_unassigned_ = None):
    mu = mu_.copy()
    V_j = V_j_.copy()
    perm_unassigned = perm_unassigned_.copy()


    C  = np.zeros(self.n, dtype= bool)
    C_i_ent = np.zeros(self.n, dtype= bool)
    C[i_ent] = 1
    B = np.ones(self.m, dtype= bool)
    L_j = np.ones(self.m, dtype= int) *(self.n + 1)
    d_j = np.ones(self.m + 1, dtype= int) * np.inf

    i_j_diffs = np.zeros([self.n, self.m])

    iter = 0
    while np.any(C == 1):
        iter += 1
        i = np.argmax(C)
        #print(f"iter {iter}, i: {i}")
        C[i] = 0
        C_i_ent[i] = 1
 
        U_ij_s = self.get_U_ij(V_j, i)[0]
        #print(U_ij_s)
     
        π_i = np.max(U_ij_s)
        #print(π_i)
        if π_i < self.lb_U:
            print("perm unassigned")
            perm_unassigned[i] = 1
            mu[i,:] = 0
            return mu, V_j, perm_unassigned
        
        
        suboptimal_js = U_ij_s < π_i 
      
        A_ij_star_max = np.max(self.A_ij[i,~suboptimal_js ])

        i_j_diffs[i, suboptimal_js] =  (π_i - U_ij_s[suboptimal_js] ) /A_ij_star_max


        d_j[:-1][suboptimal_js] = np.minimum(d_j[:-1][suboptimal_js], (π_i - U_ij_s[suboptimal_js] ) /A_ij_star_max) 
        d_j[-1] = np.minimum(  d_j[-1],   (π_i - self.lb_U) / A_ij_star_max)
      
        unissigned_optimal_js = np.where((mu.sum(0) == 0) * (π_i == U_ij_s))[0]
        
        if len(unissigned_optimal_js)> 0 :

            i_t = i
            j_t = unissigned_optimal_js[0]
      
            while i_t != i_ent :
 
                j_t_plus_1 = np.where(mu[i_t,: ]>0)[0][0]
          
                mu[i_t,: ] = 0
                mu[i_t, j_t] = 1
           
                w_i_s = np.sort( np.concatenate([self.get_U_ij(V_j, i_t)[0], [self.lb_U]]))[-2]
         
                V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)

                j_t = j_t_plus_1
                i_t = L_j[j_t]
           

            w_i_s = np.sort( np.concatenate([self.get_U_ij(V_j, i_t)[0], [self.lb_U]]))[-2]
            mu[i_t,: ] = 0
            mu[i_t, j_t] = 1

            V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)
            #print(f"j : {j}")
            #print(L_j)
            #print("augmentation")
            return mu, V_j, perm_unassigned
            
        
        for j in np.where(B * (~suboptimal_js) >0)[0]:
            
            #U_ij = self.get_U_ij(V_j, i, j)[0]
            if mu[:,j].sum() == 1:
                i_j = np.argmax(mu[:,j])
                B[j] = 0
                C[i_j] = 1
                
                L_j[j] = i if L_j[j] == self.n + 1 else None
                #print(f"j : {j}")
                #print(L_j)
                #print(f"i:{i} -> j:{j} ")


    # matrix = i_j_diffs[ :, B * (d_j[:-1] < np.inf)]  
    # masked_matrix = np.ma.masked_where(matrix <= 0, matrix)

    # argmin_pos = np.where(masked_matrix == np.min(masked_matrix))
    #argmin_pos = np.unravel_index(np.argmin(masked_matrix), masked_matrix.shape)

    
    # construct j_low's
    j_low = np.zeros(self.m, dtype= bool)
    # for i_low in np.unique(argmin_pos[0]):
    
    #     U_ij_s = self.get_U_ij(V_j, i_low)[0]
    #     π_i = np.max(U_ij_s)
    #     optimal_js = U_ij_s == π_i 
    
    #     j_low[optimal_js] = self.A_ij[i_low,optimal_js] == np.max(self.A_ij[i_low,optimal_js])
     


    return C_i_ent, B * (d_j[:-1] < np.inf), d_j, j_low

OneToOneITU.iteration = iteration

In [5]:
def interim_auction(self,C, O, j_low, r, V_j_ ):
    V_j = V_j_.copy()
    pi_ij = np.zeros([C.sum(), self.m+1])
    #print(V_j)
    V_j[j_low] += r
    #print(V_j)
    unassigned = pi_ij.sum(1) == 0
    n_unassigned = unassigned.sum()
    iter = 0
    someone_unhappy = True
  
    U_ij = np.concatenate((self.get_U_ij(V_j, C), np.ones((C.sum(),1)) * self.lb_U), axis = 1) 
    #print(U_ij.round(3))
    
    #print(U_ij.round(1))

    sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
    w_i = U_ij[np.arange(n_unassigned),sorted_w_id[:,0] ]
    π_i = np.max(U_ij, axis = 1)

    #pi_ij[:,-1] = π_i == self.lb_U 
    ij_optimal = U_ij == π_i[:, None]
    offers = ((π_i - w_i)[:,None]/ self.A_ij[C,:] ) * ij_optimal[:,:-1]


    if offers.mean() <= 0:
        #print(ij_optimal[:,:-1][:,O])
        pi_ij_O = solve_1to1(ij_optimal[:,:-1][:,O] )
        unass = pi_ij_O.sum(1) == 0
        #print("MMMM")
        #print(np.concatenate((O, [False])))
        pi_ij[:,np.concatenate((O, [False]))] = pi_ij_O
        #pi_ij[:,-1] = unass

        return pi_ij, V_j, 0







    while pi_ij.sum() < C.sum()-1 or offers[:,O].mean() > 1e-14:

        U_ij = np.concatenate((self.get_U_ij(V_j, C), np.ones((C.sum(),1)) * self.lb_U), axis = 1) 
        #print(U_ij.round(3))
        
        #print(U_ij.round(1))

        sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
        w_i = U_ij[np.arange(n_unassigned),sorted_w_id[:,0] ]
        π_i = np.max(U_ij, axis = 1)
    
        #pi_ij[:,-1] = π_i == self.lb_U 
        ij_optimal = U_ij == π_i[:, None]
        offers = ((π_i - w_i)[:,None]/ self.A_ij[C,:] ) * ij_optimal[:,:-1]

        #print(offers.round(1))
        #print(w_i)

        
        free_i = np.ones(C.sum(), dtype= bool) 
        for j in np.where(O > 0)[0]:
            if np.any(ij_optimal[:,j]>0) and len(offers[:,j][free_i])>0:
                
                best_offer = np.max(offers[:,j] [free_i] )
                i_j = np.argmax(offers[:,j] - (1- free_i)*(1e+200)- (1- ij_optimal[:,j])*(1e+200))
                #print(f"j:{j}, best i: {i_j}")
                #print(free_i)
                pi_ij[:, j] = 0 
                pi_ij[i_j, :] = 0 
                pi_ij[i_j, j] = 1 
                free_i[i_j] = 0
                V_j[j] += best_offer 

        out_offers = ij_optimal[:,-1]
        #pi_ij[:,-1] = out_offers *  free_i

       
        pi_i = np.max(self.get_U_ij(V_j, C), axis= 1)
        U_ij = np.concatenate((self.get_U_ij(V_j, C), np.ones((C.sum(),1)) * self.lb_U), axis = 1) 
        #print(U_ij.round(3))
        ij_optimal = U_ij == pi_i[:, None]
        sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
        w_i = U_ij[np.arange(n_unassigned),sorted_w_id[:,0] ]

        offers = ((pi_i - w_i)[:,None]) * ij_optimal[:,:-1]

        #print( offers[:,O].sum() )
  

        # id_i,  id_j = np.where(pi_ij[:,:-1] >0)
        # #print(np.where(pi_ij[:,:-1] >0))
        # happy_i = np.zeros(C.sum(), dtype= bool) 
        # Values = self.get_U_ij(V_j,C )
        # happy_i[id_i]  = Values[id_i, id_j] >= pi_i[id_i]

        # unassigned = np.where(pi_ij[:,-1] >0)[0]
        # happy_i[unassigned] = pi_i[unassigned] <= self.lb_U
    
        # someone_unhappy = np.any(happy_i == False)

        iter += 1 
        if iter == 100:
            break

    #print(pi_ij)
 
    pi_ij_O = solve_1to1(ij_optimal[:,:-1][:,O]   )
    #unass = pi_ij_O.sum(1) == 0
    #print("MMMM")
    #print(np.concatenate((O, [False])))
    pi_ij[:,np.concatenate((O, [False]))] = pi_ij_O
    #pi_ij[:,-1] = unass
    #print(pi_ij)

    return pi_ij, V_j, iter

OneToOneITU.interim_auction =interim_auction
        

In [1274]:
V_ex_pre

array([0., 0., 0., 0., 0.])

In [1368]:
V_ex_pre + j_low * min_overall

array([0. , 1. , 1. , 0.5, 1. ])

In [1369]:
U_ij = example_small.get_U_ij(V_ex , np.arange(example_small.n))[C_i].round(1)
print(U_ij)

[[ 2.   2.5  0.5  1.5  2.5]
 [ 1.   1.   0.  -0.2  1. ]
 [ 2.   2.   2.  -0.2  2. ]]


In [1370]:
pi_i = (example_small.get_U_ij(V_ex, np.arange(example_small.n))[C_i].round(1)).max(1)

In [1371]:
B

array([ True, False,  True,  True, False])

In [1372]:
virt_offs

array([[ 0. ,  1.5, -3. , -0.5,  1.5],
       [ 0. ,  0.5, -1. , -2.5,  0.5],
       [ 0. ,  0.5,  0. , -4.5,  0.5]])

In [1373]:
(U_ij == pi_i[:,None]) *1

array([[0, 1, 0, 0, 1],
       [1, 1, 0, 0, 1],
       [1, 1, 1, 0, 1]])

In [1374]:
solve_1to1( (U_ij == pi_i[:,None]))

array([[0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [1375]:
example_small.A_ij[C_i,:] * (U_ij == pi_i[:,None])

array([[0. , 0.5, 0. , 0. , 0.5],
       [0.5, 1. , 0. , 0. , 1. ],
       [0.5, 1. , 1. , 0. , 1. ]])

In [1376]:
U_ij == pi_i[:,None]

array([[False,  True, False, False,  True],
       [ True,  True, False, False,  True],
       [ True,  True,  True, False,  True]])

In [1377]:
w_i_out_B = np.max(example_small.get_U_ij(V_ex, np.arange(example_small.n))[C_i][:,B], axis = 1)

In [1378]:
(((U_ij - w_i_out_B[:,None])/ example_small.A_ij[C_i,:])).round(2)

array([[ 0. ,  1. , -3. , -0.5,  1. ],
       [ 0. ,  0. , -1. , -2.4,  0. ],
       [ 0. ,  0. ,  0. , -4.4,  0. ]])

In [1379]:
((U_ij - w_i_out_B[:,None])/ example_small.A_ij[C_i,:])[2,4]

0.0

In [1380]:
solve_1to1(np.array([[ True , True],
 [ True , True],
 [ True , True]]))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


array([[1, 0],
       [0, 1],
       [0, 0]])

In [1381]:
example_small.interim_auction(C_i, ~B, ~B, min_overall,V_ex_pre )

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


(array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 array([0. , 1. , 1. , 0.5, 1. ]),
 0)

In [1382]:
B

array([ True, False,  True,  True, False])

# Examples

In [1383]:
mu_ex = np.zeros([n_small,m_small])
V_ex = np.ones(example_small.m) * example_small.lb_V
perm_unassigned = np.zeros(example_small.n, dtype= bool)

In [1384]:
example_small.B_ij

array([[2, 3, 3, 3, 1],
       [2, 3, 1, 2, 3],
       [0, 0, 2, 2, 2],
       [1, 2, 1, 0, 2],
       [2, 3, 3, 0, 3]])

In [1385]:
# #C_i = np.zeros(example_small.n, dtype= bool)

# for p in range(1):

#     i_ent = np.where((mu_ex.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
#     iter_values = example_small.iteration(i_ent, mu_ex, V_ex )

#     if len(iter_values) == 2:
#         mu_ex, V_ex = iter_values

#     else:
#         C_i, B, d ,j_low= iter_values
#         print(B)
#         B_with_out = np.concatenate( (B, [True]) )
#         r = d[B_with_out]
#         #V_ex[~B] += np.min(d[B_with_out])

#         print(d)
#         print(B)
#         mu_ex[C_i,:] = 0
#         busy = np.zeros(example_small.m, dtype= bool)
#         for i in np.where(C_i > 0)[0]:
#             j_i = np.argmax( example_small.get_U_ij(V_ex, i) )
#             #print(j_i)
#             if busy[j_i] == False:
#                 mu_ex[:,j_i] = 0
#                 mu_ex[i,j_i] = 1
#                 busy[j_i] = 1


       

#         #print(B)
#         print(f"d:{np.min(d[B_with_out])}")
#         print(f"iteration total:{p}")
    
        

#     perm_unassigned = np.all(example_small.get_U_ij(V_ex , np.arange(example_small.n)) <= example_small.lb_U , axis = 1)
# mu_ex[perm_unassigned,:] = 0

In [1386]:
#C_i = np.zeros(example_small.n, dtype= bool)
for p in range(1):

    i_ent = np.where((mu_ex.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
    iter_values = example_small.iteration(i_ent, mu_ex, V_ex ,perm_unassigned)

    if len(iter_values) == 3:
        mu_ex, V_ex , perm_unassigned = iter_values

      
    else:
        #print("AAAA")
        C_i, B, d ,j_low= iter_values
        
        B_with_out = np.concatenate( (B, [True]) )
        r = d[B_with_out]

        
        U_ij = example_small.get_U_ij(V_ex, np.arange(example_small.n))[C_i]
        pi_i = np.max(U_ij, axis = 1)
        w_i_out_B = np.max(U_ij[:,B], axis = 1)

        virt_offs = ((U_ij - w_i_out_B[:,None])/ example_small.A_ij[C_i,:])

        min_off = np.min( virt_offs + 1e200 * (virt_offs <= 0), axis = 0)

        min_overall = min_off.min()

        j_low = min_off == min_overall

        V_ex_pre = V_ex.copy()
        #V_ex[~B] += np.min(d[B_with_out])
        pi_ij, V_ex, iter = example_small.interim_auction(C_i, ~B, ~B, min_overall,V_ex )

        # this should be revoming exactly one 0
        # print(pi_ij)
        #pi_ij[:,:-1][:,mu_ex[~C_i,:].sum(0) == 1 ]= 0


        mu_ex[C_i,:] = pi_ij[:,:-1]
        perm_unassigned[C_i] = pi_ij[:,-1]


    
   
# mu_ex[perm_unassigned,:] = 0

In [1387]:
V_ex

array([0., 0., 0., 0., 0.])

In [1388]:
mu_ex

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [1389]:
(example_small.get_U_ij(V_ex , np.ones(example_small.n, dtype= bool))).round(2)

array([[2., 3., 3., 3., 1.],
       [2., 3., 1., 2., 3.],
       [0., 0., 2., 2., 2.],
       [1., 2., 1., 0., 2.],
       [2., 3., 3., 0., 3.]])

In [1390]:
i_ent = np.where((mu_ex.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
example_small.iteration(i_ent, mu_ex, V_ex ,perm_unassigned)

(array([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([0., 0., 0., 0., 0.]),
 array([False, False, False, False, False]))

In [1391]:
C_i, B, d ,j_low= example_small.iteration(i_ent, mu_ex, V_ex ,perm_unassigned)

B_with_out = np.concatenate( (B, [True]) )
r = d[B_with_out]

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
j_low

array([False, False,  True, False, False])

In [ ]:
example_small.interim_auction(C_i, ~B, j_low,  np.min(r),V_ex )

KeyboardInterrupt: 

In [ ]:
pi_ij[:,:-1]

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [6]:
def coop_ITU_auction(self):
    mu_ij = np.zeros([self.n,self.m])
    V_j = np.ones(self.m) * self.lb_V
    perm_unassigned = np.zeros(self.n, dtype= bool)
    iters = 0 
    while np.sum((mu_ij.sum(1) == 0) * (~perm_unassigned)) > 0:
        iters += 1
        i_ent = np.where((mu_ij.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
        iter_values = self.iteration(i_ent, mu_ij, V_j ,perm_unassigned)
        #print(iter_values)
        if len(iter_values) == 3:
            mu_ij, V_j ,perm_unassigned = iter_values

        else:
          
            C_i, B, d ,j_low= iter_values
            
            B_with_out = np.concatenate( (B, [True]) )
            r = d[B_with_out]


            U_ij = self.get_U_ij(V_j, np.arange(self.n))[C_i]
            pi_i = np.max(U_ij, axis = 1)
            w_i_out_B = np.max(U_ij[:,B], axis = 1)

            virt_offs = ((U_ij - w_i_out_B[:,None])/ self.A_ij[C_i,:])

            min_off = np.min( virt_offs + 1e200 * (virt_offs <= 0), axis = 0)

            min_overall = min_off.min()

            j_low = min_off == min_overall


            #V_j[~B] += np.min(d[B_with_out])
            pi_ij, V_j, iter = self.interim_auction(C_i, ~B, ~B, min_overall,V_j )





            mu_ij[C_i,:] = pi_ij[:,:-1]
            perm_unassigned[C_i] = pi_ij[:,-1]


        

    #mu_ij[perm_unassigned,:] = 0



    id_i , id_j  =  np.where(mu_ij == 1)
    
    U_i = np.ones(self.n) * self.lb_U
    U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j] 
    #print(iters)
    return mu_ij, U_i , V_j




OneToOneITU.coop_ITU_auction = coop_ITU_auction

In [ ]:
# def ITU_auction(self):
#     mu_ij = np.zeros([self.n,self.m])
#     V_j = np.ones(self.m) * self.lb_V
#     perm_unassigned = np.zeros(self.n, dtype= bool)
#     iters = 0
#     # for p in range(10000):

#     while np.sum((mu_ij.sum(1) == 0) * (~perm_unassigned)) > 0:
#         iters += 1
#         i_ent = np.where((mu_ij.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
#         iter_values = self.iteration(i_ent, mu_ij, V_j )

#         if len(iter_values) == 2:
#             mu_ij, V_j = iter_values

#         else:
#             print("AAAA")
#             C_i, B, d ,j_low= iter_values
            
#             B_with_out = np.concatenate( (B, [True]) )
#             r = d[B_with_out]
#             #V_j[~B] += np.min(d[B_with_out])
#             pi_ij, V_j, happy_i, iter = self.interim_auction(C_i, ~B, j_low,  np.min(r),V_j )

#             # this should be revoming exactly one 0
#             pi_ij[:,:-1][:,mu_ij[~C_i,:].sum(0) == 1 ]= 0


#             mu_ij[C_i,:] = pi_ij[:,:-1]
#             perm_unassigned[C_i] = pi_ij[:,-1]


    
        

#     perm_unassigned = np.all(self.get_U_ij(V_j , np.arange(self.n)) <= self.lb_U , axis = 1)
#     mu_ij[perm_unassigned,:] = 0

#     id_i , id_j  =  np.where(mu_ij == 1)
    
#     U_i = np.ones(example_small.n) * example_small.lb_U
#     U_i[id_i] = example_small.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j] 
#     return mu_ij, U_i , V_j



#     # id_i , id_j  =  np.where(mu_ij == 1)
    
#     # U_i = np.ones(self.n) * self.lb_U
#     # U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]   

#     # return mu_ij,U_i, V_j, perm_unassigned * 1, iters

# OneToOneITU.ITU_auction = ITU_auction

In [23]:
n_small = 50
m_small = 50
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (5,5) , random_seed= 27, lbs =(0,0), tol = 0)

In [1427]:
# mu_ex , U_ex,  V_ex , perm_ , iters= example_small.ITU_auction()

In [24]:
mu_ex , U_ex,  V_ex  = example_small.coop_ITU_auction()

In [25]:
example_small. check_dual_feas(U_ex,V_ex, output = True)[0].sum()

0.0

In [26]:
example_small.check_primal_CS(U_ex, V_ex, mu_ex, output= True).sum()

0.0

In [27]:
((mu_ex.sum(0) == 0) * ( V_ex - example_small.lb_V)).sum() 

0.0

In [28]:
mu_ex.sum()

50.0

In [30]:
# id_i , id_j  =  np.where(mu_ex == 1)
    
# U_ex = np.ones(example_small.n) * example_small.lb_U
# U_ex[id_i] = example_small.get_U_ij(V_ex, id_i)[np.arange(len(id_i)), id_j]  

In [29]:
example_small.check_all((mu_ex , U_ex , V_ex ))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 50 over 50

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [31]:
# example_small. check_dual_feas(U_ex,V_ex, output = True)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (array([], dtype=int64), array([], dtype=int64)))

In [32]:
for iter in range(100000):
    print(iter)
    n_small = 100
    m_small = 100
    example_simul = OneToOneITU( n = n_small,m = m_small, size_params= (5,5) , random_seed= iter, lbs =(0,0), tol = 0)
    mu_sim , U_sim,  V_sim  = example_simul.coop_ITU_auction()
    

    feas_dual = example_simul. check_dual_feas(U_sim,V_sim, output = True)[0].sum() 
    CS = example_simul.check_primal_CS(U_sim, V_sim, mu_sim, output= True).sum()
    IR = ((mu_sim.sum(0) == 0) * ( V_sim - example_simul.lb_V)).sum()   
    feas = np.all(mu_sim.sum(0)<=1) * np.all(mu_sim.sum(1)<=1)

    if CS != 0 or IR != 0 or ~feas or feas_dual != 0:


        print("porcaccio")
        print(IR)
        print(CS)
        print(f"iter{iter}")
        break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [954]:
example_small.get_U_ij(V_ex, np.arange(example_simul.n)).round(2)

array([[ 0.67, -1.  , -3.  , -2.  , -0.75],
       [ 0.67, -1.  , -2.  , -2.  ,  0.25],
       [-0.33, -1.  , -2.  , -3.  ,  0.25],
       [ 0.67, -1.  , -2.  , -3.  ,  0.25],
       [-0.33, -1.  , -2.  , -3.  , -0.75]])

In [955]:
mu_ex

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.]])